In [ ]:
using Pkg
Pkg.update()
Pkg.add(["LazIO", "GeoDataFrames", "GeoFormatTypes", "Extents", "DataFrames", "GeoInterface", "GeoInterfaceRecipes", "GeoArrays", "GeoArrayOps", "SpaceLiDAR", "Plots", "Shapefile", "GeoStatsSolvers"])
Pkg.add(url="https://github.com/Deltares/PointCloudRasterizers.jl")  # package will be registered from 16-01-2023 on

# LiDAR pointclouds
In this notebook we open a small LiDAR pointcloud in the .laz format and try to classify the ground surface.

In [ ]:
using LazIO, PointCloudRasterizers, GeoInterface, GeoInterfaceRecipes, GeoDataFrames, Plots, DataFrames, GeoFormatTypes, GeoArrayOps, GeoArrays, Statistics, GeoStatsSolvers

In [ ]:
fn = "vlieland.laz"
ds = LazIO.open(fn)

In [ ]:
plot(collect(ds)[begin:100:end])

In [ ]:
df = DataFrame(ds)

In [ ]:
histogram(df.number_of_returns)

In [ ]:
GeoDataFrames.write("vlieland.gpkg", df)

In [ ]:
round(Int, filesize("vlieland.laz") / filesize("vlieland.gpkg") * 100)

# Classifying the pointcloud
Let's classify the pointcloud, specifically to find the terrain.

In [ ]:
idx = PointCloudRasterizers.index(ds, (1, 1), crs=GeoFormatTypes.EPSG(28992));

In [ ]:
plot(idx.counts, size=(1000,1000))

In [ ]:
last_return(p) = p.return_number == p.number_of_returns

In [ ]:
filter!(idx, last_return);

In [ ]:
plot(idx.counts, size=(1000,1000))

In [ ]:
min_raster = reduce(idx, reducer=minimum)

In [ ]:
plot(min_raster, c=:turbo, clim=(0,18), size=(1000,1000))

In [ ]:
max_raster, windows = pmf(GeoArrays.coalesce(min_raster, NaN))

In [ ]:
plot(max_raster, c=:turbo, clim=(0,18), size=(1000,1000))

In [ ]:
?pmf

In [ ]:
max_raster2, _ = pmf(GeoArrays.coalesce(min_raster, NaN),slope=0.1, ωₘ=35., dh₀=1.0, dhₘ=10.)

In [ ]:
plot(max_raster2, c=:turbo, clim=(0,18), size=(1000,1000))

In [ ]:
terrain(p, r) = GeoInterface.z(p) <= r

In [ ]:
filter!(idx, max_raster2, terrain);

In [ ]:
plot(idx.counts, size=(1000,1000))

In [ ]:
dtm = reduce(idx, reducer=mean)

In [ ]:
plot(dtm, size=(1000,1000), c=:turbo)

Now we just need to interpolate. We can use any Estimation Solver from GeoStats: https://juliaearth.github.io/GeoStats.jl/dev/solvers/builtin.html#Estimation

In [ ]:
GeoArrays.fill!(dtm, Kriging(:band => (maxneighbors=10,)))

In [ ]:
plot(dtm, size=(1000,1000), c=:turbo)